# Data

This notebook was the code used to generate the samples used in the final model. It also includes prototypes of the models implmented in the final not book. The most important piece of code was the loop that went over the data file and parsed out the categories of interest. This was the biggest bottleneck in modeling as running this block would take between three and five hours. By saving the results into numpy arrays, the results could be quickly accessed in the final notebook. 



In [9]:
# Hacking a non-streaming CNN Model

import cv2

from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))
# Any results you write to the current directory are saved as output. 
#KS Ask about this line

import matplotlib.pyplot as plt
%matplotlib inline 
from skimage.data import imread   # or, whatever image library you prefer
import multiprocessing as mp      # will come in handy due to the size of the data
import bson
import io
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import LabelEncoder

import time
import threading
import itertools
from tqdm import *

import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Lambda
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix

import tensorflow as tf

In [10]:
# Looking specifically over the range for the following categories, while aready numeric
# I will use label encoder to take out any sense of ordinal value 

targets = [1000003437, 1000003407, 1000003400, 1000003404, 1000003402]

In [11]:
label_encoder = LabelEncoder()
label_encoder.fit(targets)

LabelEncoder()

In [12]:
data = bson.decode_file_iter(open('/Volumes/G-DRIVE mobile/Image_Data_DSI/train.bson', 'rb'))

In [13]:
# load data and convert it into an iterable version of the data

In [14]:
it = iter(data)

In [15]:
# This is the method to stream through the test set and pull out the data 
X = []
y = []
counter=0
with tqdm(total=6728) as pbar:
    for i in range(6009411):
        piece = next(it)
        if piece.get('category_id') in targets:
            counter = counter + 1
            if counter % 500 == 0: 
                print(counter)
            img = piece.get('imgs')[0]
            pic = io.BytesIO(img.get('picture', None))
            im = imread(pic)
            im = im.astype('float32') / 255.0
            X.append(im)
#             np.array(X)

            category = piece.get('category_id', "")
            label = label_encoder.transform([category])
            y.append(label)
#             np.array(y)
            pbar.update()





  7%|▋         | 500/6728 [02:00<23:14,  4.46it/s]  

500


 15%|█▍        | 999/6728 [05:05<42:56,  2.22it/s]  

1000


 22%|██▏       | 1499/6728 [09:08<46:47,  1.86it/s]  

...

 89%|████████▉ | 5999/6728 [1:52:07<30:44,  2.53s/it]

6000


 97%|█████████▋| 6499/6728 [2:12:45<09:24,  2.46s/it]

6500


 99%|█████████▉| 6660/6728 [2:20:25<02:59,  2.64s/it]


In [56]:
len(X)


6660

In [75]:
len(y)

6660

In [57]:
X =np.array(X)

In [58]:
type(X)

numpy.ndarray

In [59]:
y = np.array(y)

In [60]:
type(y)

numpy.ndarray

In [61]:
#y = to_categorical(y)

In [77]:
yt.shape

(6280, 2)

In [62]:
with open('wine_Xtrain.npy', 'wb') as np_out:
    np.save(np_out,X)

In [63]:
with open('wine_ytrain.npy', 'wb') as np_yout:
    np.save(np_yout,y)

In [64]:
import pickle

In [ ]:
# Making these into arrays may have created a problem. Can try to convert again
# or open new notebook and run again without the array line, hopefully will be faster. 


In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2003)

X_train = X_train.reshape(X_train.shape[0], 180, 180, 3)
X_test = X_test.reshape(X_test.shape[0], 180, 180, 3)


In [83]:
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(180, 180, 3), activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(16, (4,4), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2)

Train on 4995 samples, validate on 1665 samples
Epoch 1/2
4995/4995 [==============================] - 211s 42ms/step - loss: 0.9460 - acc: 0.6593 - val_loss: 0.7449 - val_acc: 0.7357
Epoch 2/2
4995/4995 [==============================] - 202s 41ms/step - loss: 0.6015 - acc: 0.7810 - val_loss: 0.5980 - val_acc: 0.7910


In [84]:
model.save("KPS_Model_1")

In [85]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 176, 176, 32)      2432      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 88, 88, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 85, 85, 16)        8208      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 42, 42, 16)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 28224)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                903200    
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 165       
Total para

In [87]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

Train on 4995 samples, validate on 1665 samples
Epoch 1/3
4995/4995 [==============================] - 206s 41ms/step - loss: 0.4447 - acc: 0.8414 - val_loss: 0.5652 - val_acc: 0.8168
Epoch 2/3
4995/4995 [==============================] - 203s 41ms/step - loss: 0.3358 - acc: 0.8879 - val_loss: 0.5876 - val_acc: 0.8204
Epoch 3/3
4995/4995 [==============================] - 202s 40ms/step - loss: 0.2541 - acc: 0.9107 - val_loss: 0.6489 - val_acc: 0.8126


In [89]:
model.save("KPS_Model_1_5epochs")

In [ ]:
# Now will generate true out of sample data from the remaining images in the set, will
# loop through again, since there are 7898 images in the set, know there are 1,298 unseen images
# in teh remaining ~1mm images. I will re-run the loop above, since it is an iterator, it will
# pick up on data where left off before. 

In [38]:
# This is the method to stream through the test set and pull out the data 
Xt = []
yt = []
counter=0
with tqdm(total=1298) as pbar:
    for i in range(1060480):
        piece = next(it)
        if piece.get('category_id') in targets:
            counter = counter + 1
            if counter % 100 == 0: 
                print(counter)
            img = piece.get('imgs')[0]
            pic = io.BytesIO(img.get('picture', None))
            im = imread(pic)
            im = im.astype('float32') / 255.0
            Xt.append(im)
#             np.array(Xt)

            category = piece.get('category_id', "")
            label = label_encoder.transform([category])
            yt.append(label)
#             np.array(yt)
            pbar.update()



  8%|▊         | 100/1298 [00:20<03:25,  5.82it/s]

100


 15%|█▌        | 200/1298 [00:44<04:47,  3.81it/s]

200


 23%|██▎       | 299/1298 [01:10<03:29,  4.77it/s]

...

 85%|████████▍ | 1099/1298 [06:58<02:05,  1.58it/s]

1100


 92%|█████████▏| 1199/1298 [07:59<01:06,  1.48it/s]

1200


 97%|█████████▋| 1256/1298 [08:38<00:26,  1.57it/s]


In [65]:
len(yt) + len(y)

7916

In [66]:
yt = np.array(yt)

In [67]:
#yt = to_categorical(yt)

In [70]:
# Good spot check, y and T are seperate!
# left five spaces on end of next(it) to verify was at end. So now the full dataset for 
# images I want to classify saved as variables 

In [71]:
type(Xt)

list

In [72]:
Xt = np.array(Xt)

In [73]:
with open('wine_Xtest.npy', 'wb') as np_test_out:
    np.save(np_test_out, Xt)

In [74]:
with open('wine_ytest.npy', 'wb') as np_test_you:
    np.save(np_test_you, yt)

In [108]:
preds= model.predict(Xt,32)

In [109]:
preds

array([[  4.89779472e-01,   2.96259169e-02,   2.67489795e-05,
          3.84298146e-01,   9.62697566e-02],
       [  8.62724960e-01,   6.02738990e-04,   1.68416295e-06,
          8.58174681e-05,   1.36584848e-01],
       [  6.61207247e-04,   7.28846790e-05,   4.46278449e-07,
          9.99263346e-01,   2.17029537e-06],
       ..., 
       [  9.99141932e-01,   8.37529544e-04,   8.61768399e-07,
          1.83649317e-05,   1.36255176e-06],
       [  4.72475514e-02,   1.32567540e-01,   1.95268964e-04,
          7.50540793e-01,   6.94488660e-02],
       [  9.98574495e-01,   1.41062087e-03,   1.40797583e-05,
          7.64620722e-07,   9.57362687e-08]], dtype=float32)

In [113]:
#confusion_matrix(yt, preds)

In [114]:
model.fit(X, y, validation_data=(Xt, yt), epochs=5)

Train on 6660 samples, validate on 1256 samples
Epoch 1/5
6660/6660 [==============================] - 273s 41ms/step - loss: 0.3014 - acc: 0.9029 - val_loss: 0.5873 - val_acc: 0.8368
Epoch 2/5
6660/6660 [==============================] - 254s 38ms/step - loss: 0.1980 - acc: 0.9320 - val_loss: 0.5828 - val_acc: 0.8503
Epoch 3/5
6660/6660 [==============================] - 398s 60ms/step - loss: 0.1480 - acc: 0.9483 - val_loss: 0.6331 - val_acc: 0.8463
Epoch 4/5
6660/6660 [==============================] - 260s 39ms/step - loss: 0.1198 - acc: 0.9583 - val_loss: 0.7281 - val_acc: 0.8360
Epoch 5/5
6660/6660 [==============================] - 253s 38ms/step - loss: 0.0939 - acc: 0.9704 - val_loss: 0.7623 - val_acc: 0.8511


In [ ]:
# Export the prediction classes? 


In [115]:
# Building a Second Model

model2 = Sequential()
model2.add(Conv2D(16, (2, 2), input_shape=(180, 180, 3), activation='relu'))
model2.add(MaxPool2D((2,2)))
model2.add(Conv2D(32, (4,4), activation='relu'))
model2.add(MaxPool2D((2, 2)))
model2.add(Conv2D(64, (4,4), activation='relu'))
model2.add(MaxPool2D((2, 2)))
model2.add(Flatten())
model2.add(Dense(32, activation='relu'))
model2.add(Dense(y.shape[1], activation='softmax'))

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

Train on 4995 samples, validate on 1665 samples
Epoch 1/3
4995/4995 [==============================] - 147s 29ms/step - loss: 0.9028 - acc: 0.6627 - val_loss: 0.7322 - val_acc: 0.7249
Epoch 2/3
4995/4995 [==============================] - 138s 28ms/step - loss: 0.5998 - acc: 0.7846 - val_loss: 0.5555 - val_acc: 0.8000
Epoch 3/3
4995/4995 [==============================] - 136s 27ms/step - loss: 0.4511 - acc: 0.8468 - val_loss: 0.5349 - val_acc: 0.8180


In [116]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 179, 179, 16)      208       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 89, 89, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 86, 86, 32)        8224      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 43, 43, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 40, 40, 64)        32832     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 20, 20, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 25600)             0         
__________

In [117]:
model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2)

Train on 4995 samples, validate on 1665 samples
Epoch 1/2
4995/4995 [==============================] - 141s 28ms/step - loss: 0.3567 - acc: 0.8727 - val_loss: 0.5132 - val_acc: 0.8342
Epoch 2/2
4995/4995 [==============================] - 138s 28ms/step - loss: 0.2780 - acc: 0.9047 - val_loss: 0.5008 - val_acc: 0.8486


In [118]:
model2.save("Model2_5_epochs")

In [120]:
model2.fit(X, y, validation_data=(Xt, yt), epochs=5)

Train on 6660 samples, validate on 1256 samples
Epoch 1/5
6660/6660 [==============================] - 182s 27ms/step - loss: 0.3015 - acc: 0.8988 - val_loss: 0.4275 - val_acc: 0.8623
Epoch 2/5
6660/6660 [==============================] - 172s 26ms/step - loss: 0.2149 - acc: 0.9255 - val_loss: 0.4629 - val_acc: 0.8694
Epoch 3/5
6660/6660 [==============================] - 173s 26ms/step - loss: 0.1802 - acc: 0.9390 - val_loss: 0.5875 - val_acc: 0.8264
Epoch 4/5
6660/6660 [==============================] - 174s 26ms/step - loss: 0.1396 - acc: 0.9505 - val_loss: 0.5462 - val_acc: 0.8678
Epoch 5/5
6660/6660 [==============================] - 171s 26ms/step - loss: 0.1204 - acc: 0.9596 - val_loss: 0.6231 - val_acc: 0.8551


In [1]:
# Had to fix ytests changes made in seperate notebook 